In [ ]:
SYFT_VERSION = ">=0.8.2.b0,<0.9"
package_string = f'"syft{SYFT_VERSION}"'

In [ ]:
import syft as sy
import os
sy.requires(SYFT_VERSION)
from syft.service.worker.worker_image import SyftWorkerImage
from syft.service.worker.worker_pool import WorkerStatus, SyftWorker
from syft.custom_worker.config import DockerWorkerConfig

In [ ]:
# Disable inmemory worker for container stack
in_memory_workers = not os.environ.get("ORCHESTRA_DEPLOYMENT_TYPE") == "container_stack"

In [ ]:
domain  = sy.orchestra.launch(name="test-domain-1", reset=True, dev_mode=True, create_producer=True, in_memory_workers=in_memory_workers)

In [ ]:
domain_client = domain.login(email="info@openmined.org", password="changethis")

In [ ]:
nginx_dockerfile_str = """
# Use the official Nginx image as the base
FROM openmined/grid-backend:0.8.4-beta.10

RUN pip install pydicom

"""

In [ ]:
docker_config = DockerWorkerConfig(dockerfile=nginx_dockerfile_str)

In [ ]:
assert docker_config.dockerfile == nginx_dockerfile_str

In [ ]:
res = domain_client.api.services.worker_image.submit_dockerfile(docker_config=docker_config)

In [ ]:
res

In [ ]:
assert isinstance( res, sy.SyftSuccess)

In [ ]:
dockerfile_list = domain_client.images
dockerfile_list

In [ ]:
assert len(domain_client.images.get_all()) == 2

In [ ]:
workerimage = None
for image in dockerfile_list:
    if image.config.dockerfile == nginx_dockerfile_str:
        workerimage = image
        break

assert isinstance(workerimage, SyftWorkerImage)

In [ ]:
docker_tag = "openmined/test-nginx:0.7.8"
docker_build_res = domain_client.api.services.worker_image.build(uid=workerimage.id, tag=docker_tag)

In [ ]:
docker_build_res

In [ ]:
assert isinstance(docker_build_res, sy.SyftSuccess)

In [ ]:
docker_tag

In [ ]:
import subprocess

def check_image_exists(tag) -> bool:
    result = subprocess.run(['docker', 'images', '-q', tag], stdout=subprocess.PIPE)
    return result.stdout.strip() != b''

if domain.deployment_type.value == "container_stack":
    assert check_image_exists(docker_tag)

In [ ]:
image_list = domain_client.images
image_list

In [ ]:
for image in image_list:
    if image.id == workerimage.id:
        workerimage = image  # we can also index with string using the repo_with_tag format

if domain.deployment_type.value == "container_stack":
    image_list[docker_tag]
    assert image_list[docker_tag] == workerimage

workerimage

In [ ]:
import docker

def get_image_hash(tag) -> str:
    client = docker.from_env()
    try:
        image = client.images.get(tag)
        return image.id
    except docker.errors.ImageNotFound:
        return None

In [ ]:
if domain.deployment_type.value == "container_stack":
    assert workerimage.image_hash == get_image_hash(docker_tag)

In [ ]:
def get_container_id(container_name: str) -> str:
    client = docker.from_env()
    try:
        container = client.containers.get(container_name)
        return container.id
    except docker.errors.NotFound:
        return None

In [ ]:
worker_pool_name = "my_first_worker_pool"
worker_pool_res = domain_client.api.services.worker_pool.create(name=worker_pool_name, image_uid = workerimage.id, number=3)

In [ ]:
assert len(worker_pool_res) == 3

In [ ]:
for status in worker_pool_res:
    assert status.error == None
    if domain.deployment_type.value == "container_stack":
        assert status.worker.image.image_hash == get_image_hash(docker_tag)

In [ ]:
worker_pool_list = domain_client.worker_pools
worker_pool_list

In [ ]:
assert len(domain_client.worker_pools.get_all())==2
worker_pool = None
for pool in worker_pool_list:
    if pool.name == worker_pool_name:
        worker_pool = pool
        break
assert worker_pool is not None
assert len(worker_pool.workers)==3

In [ ]:
# Delete the second worker
second_worker = worker_pool.workers[1]

In [ ]:
second_worker

In [ ]:
raw_worker_logs = domain_client.api.services.worker_pool.worker_logs(
    worker_pool_id=worker_pool.id, worker_id=second_worker.id, raw=True
)
raw_worker_logs

In [ ]:
assert isinstance(raw_worker_logs, bytes)

In [ ]:
worker_logs = domain_client.api.services.worker_pool.worker_logs(
    worker_pool_id=worker_pool.id, worker_id=second_worker.id
)
worker_logs

In [ ]:
assert isinstance(worker_logs, str)

In [ ]:
worker_delete_res = domain_client.api.services.worker_pool.delete_worker(
    worker_pool_id=worker_pool.id, worker_id=second_worker.id
)

In [ ]:
worker_delete_res

In [ ]:
assert isinstance(worker_delete_res, sy.SyftSuccess)

In [ ]:
# Refetch the worker pool
# Ensure that the deleted worker's id is not present
for pool in domain_client.api.services.worker_pool.get_all():
    if pool.name == worker_pool_name:
        worker_pool = pool
assert len(worker_pool.workers) == 2
for worker in worker_pool.workers:
    assert second_worker.id != worker.id

In [ ]:
worker_pool

In [ ]:
# delete the remaining workers
for worker in worker_pool.workers:
    res =domain_client.api.services.worker_pool.delete_worker(
        worker_pool_id=worker_pool.id, worker_id= worker.id
    )
    assert isinstance(res, sy.SyftSuccess)

In [ ]:
delete_res = domain_client.api.services.worker_image.delete(workerimage.id)

In [ ]:
# Since the containers are delete, we should be able to delete the image
assert isinstance(delete_res, sy.SyftSuccess)
delete_res